In [1]:
import cv2
import numpy as np

In [7]:
def distance_l1(x, y):
    return abs(x - y)

def distance_l2(x, y):
    return (x - y) ** 2

In [8]:
def pixel_wise_matching(left_image, right_image, disparity_range, save_result=True, method="l1"):
    # Read left, right image then convert to grayscale
    left_image = cv2.imread(left_image, cv2.IMREAD_GRAYSCALE).astype(np.float32)
    right_image = cv2.imread(right_image, cv2.IMREAD_GRAYSCALE).astype(np.float32)

    # Get the shape of the images
    height, width = left_image.shape[:2]
    
    # Create a blank image to store the result
    depth_map = np.zeros((height, width), dtype=np.uint8)
    scale = 16
    max_value = 255

    for y in range(height):
        for x in range(width):
            # Find j where cost has minimum value
            disparity = 0
            cost_min = max_value
            for j in range(disparity_range):
                if method == "l1":
                    cost = max_value if (x - j) < 0 else distance_l1(int(left_image[y, x]), int(right_image[y, x - j]))
                elif method == "l2":
                    cost = max_value if (x - j) < 0 else distance_l2(int(left_image[y, x]), int(right_image[y, x - j]))

                if cost < cost_min:
                    cost_min = cost
                    disparity = j
            depth_map[y, x] = scale * disparity

    if save_result:
        print("Saving depth map...")
        if method == "l1":
            cv2.imwrite(f"pixel_wise_l1.png", depth_map)
            cv2.imwrite(f"pixel_wise_l1_color.png", cv2.applyColorMap(depth_map, cv2.COLORMAP_JET))
        elif method == "l2":
            cv2.imwrite(f"pixel_wise_l2.png", depth_map)
            cv2.imwrite(f"pixel_wise_l2_color.png", cv2.applyColorMap(depth_map, cv2.COLORMAP_JET))
    return depth_map

In [9]:
left_image = "./tsukuba/left.png"
right_image = "./tsukuba/right.png"
disparity_range = 16

pixel_wise_matching(left_image, right_image, disparity_range, save_result=True, method="l1")
pixel_wise_matching(left_image, right_image, disparity_range, save_result=True, method="l2")

Saving depth map...
Saving depth map...


array([[ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  0, 16,  0],
       [ 0,  0,  0, ..., 16, 32,  0],
       [ 0,  0,  0, ..., 64, 64,  0]], shape=(288, 384), dtype=uint8)